# API

> FastAPI endpoints for receiving video stream/frames

In [ ]:
#| default_exp api

In [ ]:
# | export

from fastapi import FastAPI, File, UploadFile
from pydantic import BaseModel, Field
from PIL import Image, ImageFilter

from fastapi.responses import StreamingResponse
from io import BytesIO
from ray import serve

app = FastAPI()

class Image_Data(BaseModel):
    input_image_path: str = Field(
        title="path of input iamge",
        description="image path that needs to be modified",
        example="/home/naqi/vid_chains/imgs/2.jpg"
    )
    output_image_path: str = Field(
        title="path of output iamge",
        description="image path where the modified image will be saved",
        example="/home/naqi/vid_chains/imgs/blue_image.jpg"
    )


@serve.deployment(route_prefix="/")
@serve.ingress(app)
class FastAPIDeployment:
    # FastAPI will automatically parse the HTTP request for us.
    @app.post("/apis/image_path")
    async def image_path(self, input_image_path: str, output_image_path: str):
        im = Image.open(input_image_path)
        blurImage = im.filter(ImageFilter.BoxBlur(20))
        blurImage.save(output_image_path)

        return f"{output_image_path}"

    @app.post("/apis/image_file")
    async def image_file(self, img: UploadFile = File(...)):
        original_image = Image.open(img.file)
        original_image = original_image.filter(ImageFilter.BoxBlur(20))
        original_image.save("/home/naqi/vid_chains/imgs/blue_image.jpg")

        filtered_image = BytesIO()
        original_image.save(filtered_image, "JPEG")
        filtered_image.seek(0)

        return StreamingResponse(filtered_image, media_type="image/jpeg")

In [ ]:
# | export

# # 2: Deploy the deployment.
serve.run(FastAPIDeployment.bind())

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()